## Install dependencies

In [ ]:
%%capture
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
%%capture
!pip install ../input/icevision08-kagcuda/loguru-0.5.3-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ../input/icevision08-kagcuda/timm-0.4.9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ../input/icevision08-kagcuda/effdet-0.2.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ../input/icevision-081/icevision-0.8.1-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ../input/icevision08-kagcuda/omegaconf-2.0.6-py3-none-any.whl -f ./ --no-index --no-deps

In [ ]:
%%capture
%mkdir -p /root/.icevision/fonts
%cp /kaggle/input/icevision08-kagcuda/SpaceGrotesk-Medium.ttf /root/.icevision/fonts/

In [ ]:
%%capture
!pip install ../input/pycocotools202/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl -f ./ --no-index --no-deps

## Loading dependencies

In [ ]:
from fastai.vision.all import *
from fastai.medical.imaging import *
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import albumentations as A
import cv2
import gdcm
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import PIL

## Converting DICOM files to JPG

In [ ]:
def Dicom2array(path):
    im = pydicom.dcmread(path)
    is_mono1 = (im.PhotometricInterpretation == "MONOCHROME1")
    im = np.array(apply_voi_lut(im.pixel_array, im))
    if is_mono1:
        im = np.amax(im) - im
    im = im - np.min(im)
    im = im / np.max(im)
    im = (im * 255).astype(np.uint8)
    return im

def Dicom2jpg(file):
    img = Dicom2array(file)        
    out_path = Path('/kaggle/working')/Path(*file.parts[3:]).with_suffix('.jpg')
    os.makedirs(out_path.parent, exist_ok=True)
    cv2.imwrite(str(out_path), img)

In [ ]:
dicom_file = get_dicom_files('../input/siim-covid19-detection/test')
o = Parallel(n_jobs=2)(delayed(Dicom2jpg)(f) for f in tqdm(dicom_file))

## Loading model

In [ ]:
%mkdir -p /root/.cache/torch/hub/checkpoints
!cp ../input/timm-pretrained-efficientnet/efficientnet/tf_efficientnet_b0_aa-827b6e33.pth /root/.cache/torch/hub/checkpoints/tf_efficientnet_b0_aa-827b6e33.pth

In [ ]:
weights = '../input/effd017/effdet017.pth'

In [ ]:
from icevision import models, show_preds, tfms, Dataset

extra_args = {}

extra_args['img_size'] = 256

model_type = models.ross.efficientdet

backbone = model_type.backbones.tf_d0

model = model_type.model(backbone=backbone(pretrained=False), num_classes=2, **extra_args) 

max_size = 256
#extra_args = {
#    'img_size': max_size
#}

#arch = 'tf_efficientdet_d0'
#num_classes = 2
#model_type = models.ross.efficientdet
#backbone = model_type.backbones.tf_d0(pretrained=False)

In [ ]:
#model = model_type.model(backbone=backbone, num_classes=2, **extra_args)

In [ ]:
#torch.load(weights, map_location=torch.device('cpu'))["model"]

In [ ]:
model.load_state_dict(torch.load(weights, map_location=torch.device('cpu'))["model"])

In [ ]:
# !rm /root/.cache/torch/hub/checkpoints/tf_efficientnet_b3_aa-84b4657e.pth

## Predicting

In [ ]:
%run ../input/icevision-functions/icevision_functions.py

In [ ]:
image_files = get_image_files('/kaggle/working')
infer_tfms = tfms.A.Adapter([A.Resize(height=max_size, width=max_size, p=1), tfms.A.Normalize()])

In [ ]:
preds = []
for file in tqdm(image_files):
    img = PIL.Image.open(file).convert('RGB')
    pred = _end2end_detect(img, infer_tfms, model, ClassMap(['opacity']), 0.05, model_type.predict)
    for d in pred:
        d['id'] = file.stem
    preds += pred

In [ ]:
preds_df = pd.DataFrame(preds)
preds_df['PredictionString'] = preds_df.apply(lambda x: f"{x['class']} {x['score']:.3f} {' '.join([str(b) for b in x['bbox']])}", axis = 1)
preds_df = preds_df[['id', 'PredictionString']].groupby('id').agg(lambda x: ' '.join(x)).reset_index()
preds_df['id'] = preds_df['id'].apply(lambda x: x + '_image')
preds_df

## Building the submission files

In [ ]:
submission_image = pd.DataFrame({
    'id': [f.stem + '_image' for f in image_files if (f.stem + '_image') not in preds_df['id'].values],
    'PredictionString': ''
})
submission_image = pd.concat([preds_df, submission_image]).sort_values('id').reset_index(drop=True)
submission_image

In [ ]:
study = list(set([f.parent.parent.name + '_study' for f in image_files]))
prediction_string = [''] * len(study)
submission_study = pd.DataFrame({
    'id': study,
    'PredictionString': prediction_string
}).sort_values('id').reset_index(drop=True)
submission_study

In [ ]:
submission = pd.concat([submission_study, submission_image])
submission.to_csv('submission.csv',index=False)
submission

In [ ]:
import shutil
shutil.rmtree('./test')